In [8]:
import xarray as xr
import fsspec
from distributed import Client
import hvplot.xarray
import numpy as np
import hvplot.xarray  # noqa: adds hvplot method to xarray objects
import holoviews as hv
import geoviews as gv
from geoviews import opts
import pandas as pd
import panel as pn
import holoviews as hv
from datetime import datetime, timedelta
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

import warnings
warnings.filterwarnings('ignore')

from IPython.core.display import display, HTML
display(HTML("""<style>
@media (min-width: 1200px) {
  .container {
    width: 98%;
  }
}

div.output_subarea{padding:.4em .4em 0 .4em;-webkit-box-flex:1;-moz-box-flex:1;box-flex:1;flex:1;max-width:100%}

</style>
<script>
    $('#appmode-leave').hide();                          // Hides the edit app button.
    $('#appmode-busy').hide();                           // Hides the kernel busy indicator.
    $('#appmode-loader').append('<h2>Loading...</h2>');  // Adds a loading message.
;</script>

"""))


In [9]:
client = Client()

In [10]:
latlngbox = [-82, -74, 44.5, 49]
climatic_period = ['1981-01-01', '2010-12-31']

In [11]:
bucket = 's3://cires-20-century-reanalysis-v3/zarr/single-levels-space'
storage_options = {'endpoint_url': 'https://s3.us-east-1.wasabisys.com'}

In [14]:
ds = xr.open_zarr(fsspec.get_mapper(bucket,
                                    client_kwargs=storage_options,
                                    anon=True),
                  consolidated=True)
# ds = xr.open_zarr('/home/slanglois/Documents/store/final/target.zarr',
#                   consolidated=True)


In [15]:
ds_large_scale = ds

ds = ds.sel(time=slice('1900-01-01', '2015-12-31'),
            longitude=slice(latlngbox[0], latlngbox[1]),
            latitude=slice(latlngbox[2], latlngbox[3]))

In [16]:
ds_sliced = ds.sel(time=slice('1979-09-13','1979-09-15')).load()
ds_ls_sliced = ds_large_scale.sel(time=slice('1979-09-13','1979-09-15')).load()

In [108]:


def plot_variable_ls(ds, ds_ls, variable, title, numbers, shared):
    da_values = ds[variable]
    da_values_ls = ds_ls[variable]
    
    if variable == 'prate':
        da_values = da_values*3600
        da_values_ls = da_values_ls*3600
    elif variable =='tmax' or variable=='tmin':
        da_values = da_values - 273.15
        da_values_ls = da_values_ls - 273.15
    
    # 
    df_da = da_values.to_dataframe().reset_index()
    gdf = gpd.GeoDataFrame(
        df_da, geometry=gpd.points_from_xy(df_da.longitude, df_da.latitude))
    gdf.crs = 4326
    gdf = gdf.to_crs(3857)
    gdf['longitude'] = gdf.geometry.x
    gdf['latitude'] = gdf.geometry.y

    # 
    labels = hv.Labels({('x', 'y'): gdf[['longitude','latitude']],
                        'text': ['{0:.0f}'.format(i) for i in df_da[variable]]},
                                               ['x', 'y'], 'text') 
       
    clim = (da_values_ls.min().values, da_values_ls.max().values)
    # 
    jet = cm.get_cmap('jet', 256)
    newcolors = jet(np.linspace(0, 1, 256))
    newcolors[70, :] = [0.9,0.9,0.9,0]
    cmap = ListedColormap(newcolors[70:225, :])
    grid_plot = da_values.hvplot(x='longitude', y='latitude', cmap=cmap,
                    title=title,
                    clim=clim,
                    alpha=0.9,
                    geo=True)
    
    contourf_plot = da_values_ls.hvplot.contourf(levels=20,
                                                 geo=True,
                                                 hover=True,
                                                 clim=clim,
                                                 alpha=0.65,
                                                 cmap=cmap,
                                                 xlim=(-84, -72),
                                                 ylim=(44,50),
                                                 width=525,
                                                 height=375,
                                                 tiles='ESRI')
    # 
#     contour_plot = da_values_ls.hvplot.contour(levels=20,
#                                                geo=True,
#                                                clim=clim,
#                                                xlim=(-84, -72),
#                                                ylim=(44,50),
#                                                alpha=1,
#                                                cmap='isolum')
    
    # 
    shared_axes = shared.value
    
    if numbers.value is True:
        overlay =  (contourf_plot*grid_plot*labels).opts(active_tools=['wheel_zoom','pan'],
                                                         toolbar = None,
                                                         shared_axes=shared_axes)
    else:
        overlay =  (contourf_plot*grid_plot).opts(active_tools=['wheel_zoom','pan'],
                                                  toolbar = None,
                                                  shared_axes=shared_axes)
    
        
    overlay.opts(
        opts.Labels(text_font_size='8.5pt', text_font_style='bold'))
    return overlay


x = pn.widgets.Select(value='1979-09-15', options=['1979-09-15','1979-09-20'], name='# de tempête')
numbers = pn.widgets.Checkbox(name='Display numbers', value=True)
shared = pn.widgets.Checkbox(name='Shared axis', value=True)
full_screen = pn.widgets.Checkbox(name='Full screen/Normal', value=True)

layout = pn.Column(
            pn.Row('## Weather Explorer (72h window)', pn.Column(x, pn.Row(numbers, shared, full_screen))),
                pn.Row(
                plot_variable_ls(ds_sliced.max('time'), ds_ls_sliced.max('time'), 'cape', 
                                 '1979-09-15 : maximum CAPE', numbers, shared),
                plot_variable_ls(ds_sliced.max('time'), ds_ls_sliced.max('time'), 'prate', 
                                 '1979-09-15 : maximum precipitation rate', numbers, shared),
                plot_variable_ls(ds_sliced.max('time'), ds_ls_sliced.max('time'), 'pr_wtr', 
                                 '1979-09-15 : maximum precipitable water', numbers, shared)
                ),
            pn.Row(
                plot_variable_ls(ds_sliced.max('time'), ds_ls_sliced.max('time'), 'tcdc', 
                                 '1979-09-15 : maximum cloud cover', numbers, shared)
                )
            )


variables = {'cape': 'maximum CAPE',
             'prate': 'maximum precipitation rate',
             'pr_wtr' : 'maximum precipitable water',
             'tcdc': 'maximum cloud cover'}


def update(event):
    end_date = x.value
    start_date = (datetime.strptime(end_date, '%Y-%m-%d') - timedelta(days=2)).strftime('%Y-%m-%d')
    ds_ls_sliced = ds_large_scale.sel(time=slice(start_date,end_date)).load()
    ds_sliced = ds_ls_sliced.sel(longitude=slice(latlngbox[0], latlngbox[1]),
                                 latitude=slice(latlngbox[2], latlngbox[3]))
    
    for i, (key, value) in enumerate(variables.items()):                
        layout[int(np.ceil((i+1)/3))][i - int(np.floor(i/3))*3].object = plot_variable_ls(ds_sliced.max('time'), ds_ls_sliced.max('time'), 
                                                 key, '{} : {}'.format(x.value, value), numbers, shared )
x.param.watch(update, 'value')
numbers.param.watch(update, 'value')
shared.param.watch(update, 'value')

def full_screen_update(event):
    pyautogui.press('F11', presses=1)

full_screen.param.watch(full_screen_update, 'value')

layout


Column
    [0] Row
        [0] Markdown(str)
        [1] Column
            [0] Select(name='# de tempête', options=['1979-09-15', ...], value='1979-09-15')
            [1] Row
                [0] Checkbox(name='Display numbers', value=True)
                [1] Checkbox(name='Shared axis', value=True)
                [2] Checkbox(name='Full screen', value=True)
    [1] Row
        [0] HoloViews(Overlay)
        [1] HoloViews(Overlay)
        [2] HoloViews(Overlay)
    [2] Row
        [0] HoloViews(Overlay)